## Definizione del Dataset

In [1]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
import statistics

class MouseDataset(Dataset):
    """
    Gestisce il caricamento dei dati multimodali (video, pupilla, comportamento, attività neurale).
    Fornisce elementi indicizzati convertiti in tensori per il training o analisi.
    """
    def __init__(self, root_dir):
        # Salva i percorsi alle diverse sottocartelle dei dati
        self.videos_dir = os.path.join(root_dir, 'videos')
        self.pupil_dir = os.path.join(root_dir, 'pupil_center')
        self.behavioral_data_dir = os.path.join(root_dir, 'behavioral_data')
        self.labels_dir = os.path.join(root_dir, 'labels')

        # Costruisce la lista ordinata degli identificativi dei file video (senza estensione)
        self.file_ids = sorted(
            [f.replace('.npy', '') for f in os.listdir(self.videos_dir) if f.endswith('.npy')],
            key=lambda x: int(x)
        )

    def __len__(self):
        # Restituisce il numero totale di esempi disponibili
        return len(self.file_ids)

    def __getitem__(self, i):
        # Recupera l'identificativo del file in base all'indice i
        file_id = self.file_ids[i]
        # Carica da disco i diversi array .npy associati allo stesso identificativo
        video = np.load(os.path.join(self.videos_dir, f'{file_id}.npy'))
        pupil_tracking_data = np.load(os.path.join(self.pupil_dir, f'{file_id}.npy'))
        behavioral_data = np.load(os.path.join(self.behavioral_data_dir, f'{file_id}.npy'))
        neural_activity = np.load(os.path.join(self.labels_dir, f'{file_id}.npy'))

        # Aggiunge una dimensione iniziale (es. canale) al video
        video = video[np.newaxis, ...]
        
        # Converte gli array numpy in tensori float; trasforma forma dei dati pupil e comportamento
        return {
            'video': torch.from_numpy(video).float(),
            'pupil_tracking_data': torch.from_numpy(pupil_tracking_data).float().transpose(1,0),
            'behavioral_data': torch.from_numpy(behavioral_data).float().transpose(1,0),
            'neural_activity': torch.from_numpy(neural_activity).float()
        }

## Caricamento dei Dataset

In [2]:
train_set = MouseDataset('/kaggle/input/viv1tdataset/viv1tdataset/train/data')
test_set = MouseDataset('/kaggle/input/viv1tdataset/viv1tdataset/test/data')

## Funzione per calcolare la risposta media tra i trial del training

**Calcolo degli Offset (6 e 25)**

* **Clip Size:** 140 frame (limite modello Sensorium).
* **Natural Movie One:** 900 frame totali ==> `900 / 140` = **6 clip**.
* **Natural Movie Three:** 3600 frame totali ==> `3600 / 140` = **25 clip**.

Trial completo = **31 clip**

In [3]:
# Calcola la media dell'attività neurale su 7 trial della stessa clip per ridurre il rumore
def calculate_mean_neural_activity_across_trials(movie_type: str , train_set: MouseDataset, clip: int):
    # Determina l'intervallo di salto tra trial a seconda del tipo di filmato
    offset = 6 if movie_type=="one" else 25

    # Verifica che il numero della clip sia nel range consentito per il tipo di filmato
    if movie_type == "one":
        assert 1 <= clip <= 6
    elif movie_type == "three":
        assert 1 <= clip <= 25
    else:
        raise ValueError(f"movie_type deve essere 'one' o 'three'")
    
    trials = []
    # Determina l'indice iniziale nel dataset di training per la clip richiesta
    start_idx = clip-1 if movie_type == "one" else clip+41
    # Recupera sequenzialmente i 7 trial corrispondenti alla stessa clip
    for i in range(7):
        trials.append(train_set[start_idx]['neural_activity'].numpy())
        start_idx += offset
    
    # Calcola la media punto a punto sui trial (asse=0) per ottenere il segnale medio
    mean_across_trials = np.mean(trials, axis=0)
    return mean_across_trials

## Costruzione dell'array Training

In [4]:
# Costruisce l'array 3D delle attività neurali medie per ciascuna clip
list_array_numpy = [] 
# Itera sui due tipi di filmati e accumula le medie delle clip
for movie_type in ["one", "three"]:
    _range = 6 if movie_type == "one" else 25
    for i in range(_range):
        list_array_numpy.append(calculate_mean_neural_activity_across_trials(movie_type, train_set, i+1))

# Impila le medie lungo un nuovo asse per ottenere shape (31, time, neuroni)
array_3d_train = np.stack(list_array_numpy, axis=0) # Shape: (31, 227, 140)

## Preparazione del Test Set (diviso in Trial 8 e Trial 9)

In [5]:
# Creiamo le liste vuote per i due trial di test
trial_8_list = []
trial_9_list = []

# Itera su tutti gli esempi del test set e separa le attività neurali
for i in range(len(test_set)):
    if i < 31:
        trial_8_list.append(test_set[i]['neural_activity'].numpy())
    else:
        trial_9_list.append(test_set[i]['neural_activity'].numpy())

# Impila i risultati
array_3d_trial8_test = np.stack(trial_8_list, axis=0)
array_3d_trial9_test = np.stack(trial_9_list, axis=0)

print(f"Trial 8 shape: {array_3d_trial8_test.shape}") # Dovrebbe essere (31, 227, 140)
print(f"Trial 9 shape: {array_3d_trial9_test.shape}") # Dovrebbe essere (31, 227, 140)

Trial 8 shape: (31, 227, 140)
Trial 9 shape: (31, 227, 140)


## Funzione di calcolo della metrica (Correlazione di Pearson media)

In [6]:
def calculate_metrics(predictions, neural_activity):  
    # Definisce il numero di frame su cui allineare le serie temporali (ultimi 66)
    min_frames = 66
    # Allinea entrambe le serie agli ultimi frame disponibili
    neural_activity_aligned = neural_activity[..., -min_frames:]
    predictions_aligned = predictions[..., -min_frames:]
    
    # Inizializza lista delle correlazioni medie per ogni esempio
    example_correlations = [] 
    
    # Itera sugli esempi (clip) allineati
    for example_idx in range(neural_activity_aligned.shape[0]):
        # Trasforma la forma per avere (time, neuroni)
        y_true_example = neural_activity_aligned[example_idx].T  # (time, neurons)
        y_pred_example = predictions_aligned[example_idx].T  # (time, neurons)
        
        correlations = [] 
        
        # Calcola la correlazione per ciascun neurone
        for neuron in range(y_true_example.shape[1]):
            true_vals = y_true_example[:, neuron]
            pred_vals = y_pred_example[:, neuron]
            
            true_var = np.var(true_vals)
            pred_var = np.var(pred_vals)
            
            # Filtra neuroni con varianza quasi nulla per stabilità numerica
            if true_var > 1e-10 and pred_var > 1e-10:
                corr = np.corrcoef(true_vals, pred_vals)[0, 1]
                if not np.isnan(corr) and np.isfinite(corr):
                    correlations.append(corr)
        
        # Calcola la media delle correlazioni per l'esempio corrente
        mean_corr_example = np.mean(correlations) if correlations else 0.0
        example_correlations.append(mean_corr_example) 
    
    # Calcola la media finale tra tutti gli esempi
    overall_mean = np.mean(example_correlations) if example_correlations else 0.0
    
    return {
        'eval_average_single_trial_correlation': overall_mean
    }

## Calcolo finale della Baseline

In [7]:
# Calcola la metrica di correlazione per i due trial di test e stampa la baseline
risultati = [
    calculate_metrics(array_3d_train, array_3d_trial8_test)["eval_average_single_trial_correlation"],
    calculate_metrics(array_3d_train, array_3d_trial9_test)["eval_average_single_trial_correlation"]
]

print(f"Correlazioni medie per ciascun trial di test: {risultati}")
print(f"Baseline (media su entrambi i trial): {statistics.mean(risultati)}")

Correlazioni medie per ciascun trial di test: [np.float64(0.007475139404624605), np.float64(0.039593989755850795)]
Baseline (media su entrambi i trial): 0.0235345645802377
